In [ ]:
!pip install clearml
!pip install torch
!pip install torchinfo
!pip install torchmetrics
!pip install git+https://github.com/Yikai-Liao/symusic
!pip install git+https://github.com/Natooz/MidiTok
!pip install scikit-learn



In [3]:
from train import train_model

train_model()

Running on device: cuda
Loaded tokenizer:
Special tokens: ['PAD_None', 'BOS_None', 'EOS_None', 'MASK_None']
Special tokens ids: [0, 1, 2, 3]
ClearML Task: created new task id=f783b289277745adb1ae60684e319bb0
2024-08-22 06:23:52,176 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: https://app.clear.ml/projects/c282730e9b304dab984abd9d1dea0d29/experiments/f783b289277745adb1ae60684e319bb0/output/log
Loaded model:
TransformerModel(
  (embedding): Embedding(10000, 256, padding_idx=0)
  (transformer_layers): ModuleList(
    (0-3): 4 x TransformerBlock(
      (mha): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
      )
      (ffn): Sequential(
        (0): Linear(in_features=256, out_features=1024, bias=True)
        (1): ReLU()
        (2): Linear(in_features=1024, out_features=256, bias=True)
      )
      (layernorm1): LayerNorm((256,), eps=1e-06, elementwise_affine=True)
      (l

/usr/local/lib/python3.11/dist-packages/miditok/tokenizations/remi.py:77: UserWarning:

Attribute controls are not compatible with 'config.one_token_stream_for_programs' and multi-vocabulary tokenizers. Disabling them from the config.



ValueError: Could not find Dataset project/name/version ('GigMate', 'LakhMidiCleanDatasetFull', '1.0.0')

ClearML Monitor: Could not detect iteration reporting, falling back to iterations as seconds-from-start


In [ ]:
torch.save(model.state_dict(), MODEL_FILE_NAME)

In [95]:
import torch
import numpy as np
import math

def sample_from_logits(logits, temperature):
    # If temp is 0 then next_token is the argmax of logits
    if temperature == 0.0:
        next_token = torch.argmax(logits, dim=1)
    
    # If temp is not 0 then next_token is sampled out of logits
    else:
        logits = logits / temperature
        next_token = torch.multinomial(torch.softmax(logits, dim=1), num_samples=1)
    return next_token

def predict_next_note(model, input_sequence, temperature=0, past_key_values=None):
    inp = torch.tensor(input_sequence).unsqueeze(0)
    with torch.no_grad():
        outputs = model(inp)
        logits, key_values = model(inp, past_key_values=past_key_values, use_cache=True)
    predicted_tokens = sample_from_logits(logits[0], temperature)
    next_token = predicted_tokens[-1].squeeze()
    return next_token, key_values

entry = next(enumerate(train_loader))

input_sequence = entry[1]['input_ids'][0]  # Example input sequence
predicted_note = predict_next_note(get_model(), input_sequence, 0.2)
print('predicted', predicted_note)

predicted (tensor(8032), [(tensor([[[1.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.4268, 0.5732, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.4591, 0.2299, 0.3111,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.0013, 0.0015, 0.0017,  ..., 0.0017, 0.0000, 0.0000],
         [0.0012, 0.0022, 0.0019,  ..., 0.0017, 0.0016, 0.0000],
         [0.0018, 0.0011, 0.0011,  ..., 0.0018, 0.0019, 0.0022]]]), tensor([[[ 0.3546, -1.5463, -0.4309,  ...,  0.1615,  0.3449,  0.2542],
         [-0.1645,  1.0408,  1.5975,  ...,  0.8508, -0.8055,  0.7852],
         [ 0.6472, -0.3673, -0.4719,  ..., -0.3591, -0.1484, -0.5040],
         ...,
         [-1.5395, -0.7208,  0.2643,  ..., -1.1074,  0.3187,  0.7764],
         [ 0.2862,  0.8020,  0.5028,  ..., -0.5935,  0.2473, -0.8472],
         [ 0.0792, -0.2353,  0.6726,  ...,  1.6569, -1.1159,  1.4272]]])), (tensor([[[1.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.5132, 0.4868, 0.0000,  ..., 0.0000, 0.0000, 

In [98]:
input_sequence = entry[1]['input_ids'][0][250:]
output_sequence = torch.tensor(input_sequence)
next_sequence = output_sequence[-min(len(output_sequence), max_seq_len) - 1:]
last_note = len(next_sequence) - 1

next_note = -1
i = 0
key_values = None
while i < 100 and next_note != 2:  # 2 is the end of sequence code
    next_note, key_values = predict_next_note(get_model(), next_sequence, temperature=0.2, past_key_values=key_values)
    meaning = ''
    try:
        sequence = TokSequence(ids=[next_note.item()], are_ids_encoded=True)
        tokenizer.decode_token_ids(sequence)
        tokenizer.complete_sequence(sequence)
        meaning = sequence.tokens
    except Exception as e:
        print('Error', e)
    print(f'token {i}: {next_note}, meaning: {meaning}')
    output_sequence = torch.cat([output_sequence, next_note.unsqueeze(0)], 0)
    next_sequence = torch.cat([next_sequence[1:], next_note.unsqueeze(0)], 0)
    i += 1

print(f'output: {output_sequence}')
print(f'output shape: {output_sequence.shape}')

/usr/local/lib/python3.11/dist-packages/torch/utils/_device.py:77: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return func(*args, **kwargs)


AttributeError: 'tuple' object has no attribute 'shape'

In [40]:

from miditok import TokSequence
def convert_to_midi(predicted_notes):
    # Decode BPE
    #return tokenizer.decode_token_ids(predicted_notes)
    return tokenizer.decode(predicted_notes)

def get_sequence(prediction):
    token_ids = prediction.numpy().tolist()
    sequence = TokSequence(ids=token_ids, are_ids_encoded=True)
    return sequence

sequence = get_sequence(output_sequence)
print(sequence)
midi_path = convert_to_midi(sequence)
print(midi_path)
midi_path.dump_midi('output.mid')

sequence = get_sequence(input_sequence)
print(sequence)
midi_path = convert_to_midi(sequence)
print(midi_path)
midi_path.dump_midi('output_original.mid')

TokSequence(tokens=[], ids=[1214, 2241, 1131, 733, 783, 1805, 975, 1874, 788, 1805, 2840, 786, 1880, 724, 3510, 733, 4182, 705, 8713, 3608, 786, 196, 943, 2707, 567, 1178, 1227, 609, 2496, 115, 1214, 590, 2871, 6489, 943, 114, 1214, 1942, 1619, 1432, 904, 1238, 788, 2200, 783, 2200, 2398, 611, 3569, 1787, 1642, 829, 1739, 8005, 7684, 1703, 113, 1214, 605, 4077, 611, 1103, 1453, 592, 4243, 101, 1214, 1992, 897, 3835, 3712, 113, 1214, 4571, 2398, 663, 659, 588, 4077, 663, 208, 967, 5628, 114, 1214, 1885, 1619, 840, 904, 798, 678, 996, 806, 1106, 2624, 663, 4155, 664, 1642, 724, 2369, 611, 869, 2067, 3323, 2290, 114, 1214, 574, 3053, 786, 6693, 6112, 967, 4271, 115, 1214, 1629, 1619, 1391, 904, 2052, 678, 1805, 806, 2052, 2398, 786, 4406, 611, 3569, 724, 2369, 5057, 1178, 1227, 609, 3835, 4271, 2496, 116, 1214, 600, 4077, 786, 3221, 5053, 869, 3323, 114, 1214, 2058, 2624, 5231, 3835, 1920, 2703, 3343, 116, 1214, 605, 3053, 696, 1784, 1325, 4483, 967, 120, 1214, 580, 571, 2394, 1533, 837, 